<a href="https://colab.research.google.com/github/Hamza-Ali0237/PyTorch-Projects/blob/main/Intermediate/PyTorch-FineGrained-TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Grained Transfer Learning

In [ ]:
!pip install torchmetrics

In [ ]:
import torch
from torch import nn
import torchvision as tv
import torchmetrics as tm
from torchvision import transforms
from torchvision.datasets import Flowers102
from torch.utils.data import Dataset, DataLoader

import random
import numpy as np
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt

# Configuration

In [ ]:
conf_dict = {
    # Reproducibility
    "seed": 42,

    # Computing Device
    "device": "cuda" if torch.cuda.is_available() else "cpu",

    # Data Settings
    "root_dir": "./data",
    "image_size": 224,
    "batch_size": 32,
    "num_workers": 2,
    "pin_memory": True,

    # Model Settings
    "architecture": "resnet50",   # Backbone Architecture
    "pretrained": True,
    "num_classes": 102,

    # Training Hyperparameters
    "epochs": 25,
    "lr_head": 1e-3,              # LR for classifier head
    "lr_backbone": 1e-4,          # Lower LR for fine-tuning backbone
    "weight_decay": 1e-4,
    "label_smoothing": 0.0,

    # Scheduler
    "scheduler": "onecycle",

    # Imbalance Handling
    "use_class_weights": True,
    "use_weighted_sampler": False,

    # Logging & Saving
    "log_every_n_steps": 20,
    "save_best_by": "val_loss",
    "checkpoint_dir": "No checkpoints for now"
}

print(conf_dict)

In [ ]:
# Set seed for reproducibility
SEED = conf_dict["seed"]

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
  torch.cuda.manual_seed(SEED)
  torch.cuda.manual_seed_all(SEED)

# Ensure deterministic behavior
## for speed over strict determinism, set deterministic=False and benchmark=True
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False